## Data Engineering

In [ ]:
!pip install imblearn xgboost torch

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV

# For oversampling
from imblearn.over_sampling import SMOTE

# For models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

# For neural network (PyTorch)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# Initialize lists and dictionaries for model names and metrics
model_names = ["Logistic_Regression", "Decision_Tree", "Random_Forest", "XGBoost", 
               "SVC", "KNN", "Naive_Bayes", "Neural_Network", "Gradient_Boosting", "Voting_Classifier", "KMeans"]

metrics_train = {}
metrics_test = {}
for model in model_names:
    for metric in ["accuracy", "precision", "recall", "f1_score"]:
        metrics_train[f"{metric}_{model}"] = None
        metrics_test[f"{metric}_{model}"] = None
    for metric in ["accuracy", "precision", "recall", "f1_score"]:
        metrics_test[f"{metric}_kfold_{model}"] = None

models = []      # To store fitted models
models_name = [] # To store their names

In [ ]:
# Load dataset and perform basic cleaning/feature engineering
data = pd.read_csv("test/over_features.csv")
data = data.iloc[:10000]  # Limit for speed

# Check and drop missing values
print("Missing values in data:")
print(data.isnull().sum())
data.dropna(inplace=True)

# Remove identifier column
data = data.drop(columns=["match_id"])

# Feature Engineering: add enhanced features
data["pressure_index"] = data["dot_ball_pressure"] * data["required_desired_run_rate"]
data["wicket_pressure"] = data["number_of_wickets_lost"] * data["required_desired_run_rate"]
data["late_over_flag"] = (data["over"] > 15).astype(int)
data["bowler_pressure"] = data["current_bowler_economy"] * (data["bowler_wickets_in_match"] + 1)
data["aggressiveness_index"] = data["striker_strike_rate"] * (data["striker_boundaries_hit"] + 1)

print("Number of rows:", len(data))
print("Columns:", data.columns.tolist())

# Encode categorical columns
team_mapping = {team: idx for idx, team in enumerate(data["team"].unique())}
data["team_encoded"] = data["team"].map(team_mapping)
phase_mapping = {phase: idx for idx, phase in enumerate(data["match_phase"].unique())}
data["match_phase_encoded"] = data["match_phase"].map(phase_mapping)

# Drop original categorical columns and unused features
data_encoded = data.drop(columns=["team", "match_phase", "striker_boundaries_hit", "total_overs_completed"])

In [ ]:
# Train-test split and feature scaling
train_data, test_data = train_test_split(data_encoded, test_size=0.2, random_state=42)

X_train = train_data.drop(columns=["wicket_next_over"])
y_train = train_data["wicket_next_over"]
X_test = test_data.drop(columns=["wicket_next_over"])
y_test = test_data["wicket_next_over"]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# -------------------------------
# Apply SMOTE only on training data
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)
X_train_scaled = X_train_balanced
y_train = y_train_balanced

## Feature Selection & Visualization

In [ ]:
# Lasso Regression for feature importance
alphas = np.logspace(-3, 1, 100)
lasso = Lasso(max_iter=1000)
grid = GridSearchCV(lasso, param_grid={"alpha": alphas}, cv=5, scoring="r2")
grid.fit(X_train_scaled, y_train)

best_lasso = grid.best_estimator_
lasso_coef = best_lasso.coef_

plt.figure(figsize=(10, 5))
plt.bar(X_train.columns, lasso_coef, color='blue')
plt.axhline(0, color='black', linewidth=0.8, linestyle='--')
plt.xlabel("Features")
plt.ylabel("Coefficient Value")
plt.title("Lasso Regression Feature Importance")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print("Lasso Feature Coefficients:", dict(zip(X_train.columns, lasso_coef)))

In [ ]:
# Correlation Heatmap
correlation_matrix = X_train.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Random Forest Feature Importance
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)
feature_importances = rf.feature_importances_
importance_df = pd.DataFrame({
    'Feature': X_train.columns,  
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)
print("Random Forest Feature Importances:")
print(importance_df)

plt.figure(figsize=(10,6))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel('Feature Importance')
plt.title('Random Forest Feature Importance')
plt.show()

## Model Training Functions

In [ ]:
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    return accuracy, precision, recall, f1

def evaluate_model(y_train_true, y_train_pred, y_test_true, y_test_pred, model_name, model):
    # Store model and name globally
    models.append(model)
    models_name.append(model_name)
    
    # Calculate metrics and store
    accuracy_train, precision_train, recall_train, f1_train = calculate_metrics(y_train_true, y_train_pred)
    accuracy_test, precision_test, recall_test, f1_test = calculate_metrics(y_test_true, y_test_pred)
    
    metrics_train[f"accuracy_{model_name}"] = accuracy_train
    metrics_train[f"precision_{model_name}"] = precision_train
    metrics_train[f"recall_{model_name}"] = recall_train
    metrics_train[f"f1_score_{model_name}"] = f1_train

    metrics_test[f"accuracy_{model_name}"] = accuracy_test
    metrics_test[f"precision_{model_name}"] = precision_test
    metrics_test[f"recall_{model_name}"] = recall_test
    metrics_test[f"f1_score_{model_name}"] = f1_test
    
    cross_validate_model(model, model_name, X_train_scaled, y_train)

def cross_validate_model(model, model_name, X, y, n_splits=5):
    average_metrics = {"accuracy": 0, "precision": 0, "recall": 0, "f1_score": 0}
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    if isinstance(X, np.ndarray):
        X_df = pd.DataFrame(X, columns=X_train.columns)
    else:
        X_df = X.copy()
    if isinstance(y, np.ndarray):
        y_series = pd.Series(y)
    else:
        y_series = y.copy()
    
    k_folds = 0
    for train_index, test_index in skf.split(X_df, y_series):
        k_folds += 1
        X_train_cv = X_df.iloc[train_index]
        X_test_cv = X_df.iloc[test_index]
        y_train_cv = y_series.iloc[train_index]
        y_test_cv = y_series.iloc[test_index]
        
        model.fit(X_train_cv, y_train_cv)
        y_pred_cv = model.predict(X_test_cv)
        acc, prec, rec, f1 = calculate_metrics(y_test_cv, y_pred_cv)
        average_metrics["accuracy"] += acc
        average_metrics["precision"] += prec
        average_metrics["recall"] += rec
        average_metrics["f1_score"] += f1
        
    metrics_test[f"accuracy_kfold_{model_name}"] = average_metrics["accuracy"] / k_folds
    metrics_test[f"precision_kfold_{model_name}"] = average_metrics["precision"] / k_folds
    metrics_test[f"recall_kfold_{model_name}"] = average_metrics["recall"] / k_folds
    metrics_test[f"f1_score_kfold_{model_name}"] = average_metrics["f1_score"] / k_folds

def train_and_evaluate_model(model, model_name):
    model.fit(X_train_scaled, y_train)
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)
    evaluate_model(y_train, y_train_pred, y_test, y_test_pred, model_name, model)

## Train Various Models

In [ ]:
# Logistic Regression
model = LogisticRegression(multi_class='multinomial', max_iter=1000)
train_and_evaluate_model(model, "Logistic_Regression")

In [ ]:
# Decision Tree
model = DecisionTreeClassifier(random_state=42)
train_and_evaluate_model(model, "Decision_Tree")

In [ ]:
# Random Forest
model = RandomForestClassifier(random_state=42)
train_and_evaluate_model(model, "Random_Forest")

In [ ]:
# Gradient Boosting
model = GradientBoostingClassifier(random_state=42)
train_and_evaluate_model(model, "Gradient_Boosting")

In [ ]:
# SVC
model = SVC(random_state=42)
train_and_evaluate_model(model, "SVC")

In [ ]:
# KMeans
model = KMeans(n_clusters=2, random_state=42)
train_and_evaluate_model(model, "KMeans")

In [ ]:
# KNN
model = KNeighborsClassifier(n_neighbors=5)
train_and_evaluate_model(model, "KNN")

In [ ]:
# Naive Bayes
model = GaussianNB()
train_and_evaluate_model(model, "Naive_Bayes")

In [ ]:
# XGBoost
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
train_and_evaluate_model(model, "XGBoost")

In [ ]:
# Voting Classifier
model = VotingClassifier(estimators=[
    ('logistic', LogisticRegression(multi_class='multinomial', max_iter=1000)),
    ('tree', DecisionTreeClassifier(random_state=42)),
    ('forest', RandomForestClassifier(random_state=42)),
    ('boosting', GradientBoostingClassifier(random_state=42)),
    ('xgboost', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42))
], voting='soft')
train_and_evaluate_model(model, "Voting_Classifier")

## Neural Network Model (PyTorch)

In [ ]:
# Define the neural network model
num_features = X_train.shape[1]
class ImprovedWicketPredictor(nn.Module):
    def __init__(self, input_dim):
        super(ImprovedWicketPredictor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 1)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(p=0.5)

        self.batch_norm1 = nn.BatchNorm1d(64)
        self.batch_norm2 = nn.BatchNorm1d(32)
        self.batch_norm3 = nn.BatchNorm1d(16)
        self.batch_norm4 = nn.BatchNorm1d(8)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.batch_norm1(x)
        x = self.dropout(x)

        x = self.relu(self.fc2(x))
        x = self.batch_norm2(x)
        x = self.dropout(x)

        x = self.relu(self.fc3(x))
        x = self.batch_norm3(x)
        x = self.dropout(x)

        x = self.relu(self.fc4(x))
        x = self.batch_norm4(x)
        x = self.fc5(x)
        x = self.sigmoid(x)
        return x

def NN_predict(X_train_df, X_test_df, Y_train_df, Y_test_df):
    X_train_tensor = torch.tensor(X_train_df.to_numpy(), dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test_df.to_numpy(), dtype=torch.float32)
    y_train_tensor = torch.tensor(Y_train_df.to_numpy(), dtype=torch.float32).view(-1, 1)
    y_test_tensor = torch.tensor(Y_test_df.to_numpy(), dtype=torch.float32).view(-1, 1)

    train_data = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

    input_dim = X_train_df.shape[1]
    model_nn = ImprovedWicketPredictor(input_dim)

    criterion = nn.BCELoss()
    optimizer = optim.AdamW(model_nn.parameters(), lr=0.001)

    epochs = 100
    best_accuracy = 0
    patience = 10
    no_improvement = 0

    for epoch in range(epochs):
        model_nn.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model_nn(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        with torch.no_grad():
            model_nn.eval()
            y_train_pred = model_nn(X_train_tensor).round()
            accuracy_val = accuracy_score(y_train_tensor.numpy(), y_train_pred.numpy())
            if (epoch + 1) % 10 == 0:
                print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Training Accuracy: {accuracy_val:.4f}')
        if accuracy_val > best_accuracy:
            best_accuracy = accuracy_val
            no_improvement = 0
        else:
            no_improvement += 1
        if no_improvement >= patience:
            print("Early stopping triggered.")
            break

    model_nn.eval()
    with torch.no_grad():
        y_test_pred = model_nn(X_test_tensor).round()
    return y_test_pred

def cross_validate_NN(model_name, X, y, n_splits=5):
    average_metrics = {"accuracy": 0, "precision": 0, "recall": 0, "f1_score": 0}
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X, columns=X_train.columns)
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    
    k_folds = 0
    for train_index, test_index in skf.split(X, y):
        k_folds += 1
        X_train_cv = X.iloc[train_index]
        X_test_cv = X.iloc[test_index]
        Y_train_cv = y.iloc[train_index]
        Y_test_cv = y.iloc[test_index]
        
        y_pred_cv = NN_predict(X_train_cv, X_test_cv, Y_train_cv, Y_test_cv)
        y_pred_binary = (y_pred_cv > 0.5).astype(int)
        acc, prec, rec, f1 = calculate_metrics(Y_test_cv, y_pred_binary)
        average_metrics["accuracy"] += acc
        average_metrics["precision"] += prec
        average_metrics["recall"] += rec
        average_metrics["f1_score"] += f1
    
    metrics_test[f"accuracy_kfold_{model_name}"] = average_metrics["accuracy"] / k_folds
    metrics_test[f"precision_kfold_{model_name}"] = average_metrics["precision"] / k_folds
    metrics_test[f"recall_kfold_{model_name}"] = average_metrics["recall"] / k_folds
    metrics_test[f"f1_score_kfold_{model_name}"] = average_metrics["f1_score"] / k_folds

# Evaluate Neural Network on full data (for demonstration)
X_all_data = data_encoded.drop(columns=["wicket_next_over"])
Y_all_data = data_encoded["wicket_next_over"]
y_pred_nn = NN_predict(X_all_data, X_all_data, Y_all_data, Y_all_data)
acc_nn, prec_nn, rec_nn, f1_nn = calculate_metrics(Y_all_data, y_pred_nn)
print(f'Neural Network on All Data: Accuracy: {acc_nn*100:.2f}%, Precision: {prec_nn*100:.2f}%, Recall: {rec_nn*100:.2f}%, F1 Score: {f1_nn*100:.2f}%')

cross_validate_NN("Neural_Network", X_all_data, Y_all_data, n_splits=5)
print("Neural Network Cross-validation metrics:")
for metric in ["accuracy", "precision", "recall", "f1_score"]:
    print(f"{metric}_Neural_Network: {metrics_test[f'{metric}_kfold_Neural_Network']:.4f}")

# Evaluate NN on training set
X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
y_pred_train_nn = NN_predict(X_train_df, X_train_df, y_train, y_train)
acc_train_nn, prec_train_nn, rec_train_nn, f1_train_nn = calculate_metrics(y_train, y_pred_train_nn)
print(f'Neural Network on Train Data: Accuracy: {acc_train_nn*100:.2f}%, Precision: {prec_train_nn*100:.2f}%, Recall: {rec_train_nn*100:.2f}%, F1 Score: {f1_train_nn*100:.2f}%')
metrics_train["accuracy_Neural_Network"] = acc_train_nn
metrics_train["precision_Neural_Network"] = prec_train_nn
metrics_train["recall_Neural_Network"] = rec_train_nn
metrics_train["f1_score_Neural_Network"] = f1_train_nn

# Evaluate NN on test set
X_test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)
y_pred_test_nn = NN_predict(X_test_df, X_test_df, y_test, y_test)
acc_test_nn, prec_test_nn, rec_test_nn, f1_test_nn = calculate_metrics(y_test, y_pred_test_nn)
metrics_test["accuracy_Neural_Network"] = acc_test_nn
metrics_test["precision_Neural_Network"] = prec_test_nn
metrics_test["recall_Neural_Network"] = rec_test_nn
metrics_test["f1_score_Neural_Network"] = f1_test_nn
print(f'Neural Network on Test Data: Accuracy: {acc_test_nn*100:.2f}%, Precision: {prec_test_nn*100:.2f}%, Recall: {rec_test_nn*100:.2f}%, F1 Score: {f1_test_nn*100:.2f}%')

## Updated Hyperparameter Tuning to Fix Overfitting

In [ ]:
def under_or_overfitting(model, adjustment_type, model_name):
    """
    Adjust model hyperparameters to address overfitting or underfitting.
    For most models, overfitting => reduce complexity (sign = -1)
    and underfitting => increase complexity (sign = +1).
    For KNN, the adjustment is reversed.
    """
    # For most models:
    sign = -1 if adjustment_type == "overfitting" else 1

    try:
        params = model.get_params()
    except:
        return model

    # GENERAL ADJUSTMENTS
    if 'max_iter' in params and (params['max_iter'] + (100 * sign) > 0):
        model.set_params(max_iter=params['max_iter'] + (100 * sign))
        
    if 'penalty' in params:
        if adjustment_type == "overfitting":
            model.set_params(penalty='l1' if params['penalty'] == 'l2' else 'l2')

    # LOGISTIC REGRESSION: Adjust C
    if model_name == "Logistic_Regression" and 'C' in params:
        new_C = params['C'] + (0.1 * sign)
        if new_C > 0:
            model.set_params(C=new_C)

    # SVC adjustments
    if model_name == "SVC":
        if 'kernel' in params:
            kernels = ['linear', 'rbf', 'poly', 'sigmoid']
            current_kernel = params['kernel']
            next_index = (kernels.index(current_kernel) + sign) % len(kernels)
            model.set_params(kernel=kernels[next_index])
        if 'gamma' in params and isinstance(params['gamma'], (int, float)):
            new_gamma = params['gamma'] + (0.01 * sign)
            if new_gamma > 0:
                model.set_params(gamma=new_gamma)

    # TREE-BASED MODELS: Decision Tree, Random Forest, Gradient Boosting
    if model_name in ["Decision_Tree", "Random_Forest", "Gradient_Boosting"]:
        if 'max_depth' in params:
            depth = params['max_depth'] if params['max_depth'] is not None else 5
            new_depth = depth + (3 * sign)
            if new_depth > 0:
                model.set_params(max_depth=new_depth)
        if 'min_samples_split' in params:
            val = params['min_samples_split'] if params['min_samples_split'] is not None else 2
            new_val = val + (2 * sign)
            if new_val > 0:
                model.set_params(min_samples_split=new_val)
        if 'min_samples_leaf' in params:
            val = params['min_samples_leaf'] if params['min_samples_leaf'] is not None else 1
            # For overfitting, increase min_samples_leaf
            new_val = val + (2 if adjustment_type == "overfitting" else -2)
            if new_val < 1:
                new_val = 1
            model.set_params(min_samples_leaf=new_val)
        if 'max_features' in params:
            val = params['max_features']
            if isinstance(val, (int, float)):
                model.set_params(max_features=val + (1 * sign))
        if 'max_leaf_nodes' in params:
            val = params['max_leaf_nodes'] if params['max_leaf_nodes'] is not None else 10
            new_val = val + (5 * sign)
            if new_val > 0:
                model.set_params(max_leaf_nodes=new_val)
        if 'criterion' in params:
            options = ['gini', 'entropy', 'log_loss']
            if params['criterion'] in options:
                next_index = (options.index(params['criterion']) + sign) % len(options)
                model.set_params(criterion=options[next_index])
        if 'splitter' in params:
            splitters = ['best', 'random']
            current = params['splitter']
            model.set_params(splitter=splitters[(splitters.index(current) + sign) % len(splitters)])
        if 'bootstrap' in params:
            model.set_params(bootstrap=not params['bootstrap'])
        if 'n_estimators' in params:
            val = params['n_estimators'] if params['n_estimators'] is not None else 100
            new_val = val + (10 * sign)
            if new_val > 0:
                model.set_params(n_estimators=new_val)

    # KNN adjustments (reverse the sign for overfitting: increase neighbors)
    if model_name == "KNN":
        knn_sign = 1 if adjustment_type == "overfitting" else -1
        if 'n_neighbors' in params:
            new_neighbors = params['n_neighbors'] + (1 * knn_sign)
            if new_neighbors > 0:
                model.set_params(n_neighbors=new_neighbors)
        if 'weights' in params:
            options = ['uniform', 'distance']
            model.set_params(weights=options[(options.index(params['weights']) + knn_sign) % len(options)])
        if 'algorithm' in params:
            algorithms = ['auto', 'ball_tree', 'kd_tree', 'brute']
            model.set_params(algorithm=algorithms[(algorithms.index(params['algorithm']) + knn_sign) % len(algorithms)])

    # KMeans adjustments
    if model_name == "KMeans":
        if 'n_clusters' in params:
            new_clusters = params['n_clusters'] + (1 * sign)
            if new_clusters > 1:
                model.set_params(n_clusters=new_clusters)
        if 'init' in params:
            inits = ['k-means++', 'random']
            model.set_params(init=inits[(inits.index(params['init']) + sign) % len(inits)])

    # XGBoost adjustments
    if model_name == "XGBoost":
        if 'subsample' in params:
            val = params['subsample'] if params['subsample'] is not None else 1.0
            new_subsample = val + (0.05 * sign)
            if 0 < new_subsample <= 1:
                model.set_params(subsample=new_subsample)
        if 'colsample_bytree' in params:
            val = params['colsample_bytree'] if params['colsample_bytree'] is not None else 1.0
            new_val = val + (0.05 * sign)
            if 0 < new_val <= 1:
                model.set_params(colsample_bytree=new_val)
        if 'gamma' in params:
            val = params['gamma'] if params['gamma'] is not None else 0.0
            new_gamma = val + (0.1 * sign)
            if new_gamma >= 0:
                model.set_params(gamma=new_gamma)
                
    return model

def hyperparameter_tuning(metrics_train, metrics_test, models, models_name, max_tuning_iterations=10):
    """
    For each model, if test accuracy is low (< 0.8) or if training is much higher than test,
    adjust hyperparameters to reduce overfitting or increase model complexity.
    """
    for i in range(len(models)):
        model = models[i]
        model_name = models_name[i]
        
        # Initial evaluation
        model.fit(X_train_scaled, y_train)
        y_train_pred = model.predict(X_train_scaled)
        y_test_pred = model.predict(X_test_scaled)
        current_train_acc = accuracy_score(y_train, y_train_pred)
        current_test_acc = accuracy_score(y_test, y_test_pred)
        print(f"Initial {model_name}: Train Accuracy={current_train_acc:.4f}, Test Accuracy={current_test_acc:.4f}")
        
        iteration = 0
        improved = True
        while iteration < max_tuning_iterations and improved:
            old_train_acc = current_train_acc
            old_test_acc = current_test_acc
            
            # Overfitting: high train acc but low test acc
            if current_train_acc > 0.8 and current_test_acc < 0.8:
                model = under_or_overfitting(model, "overfitting", model_name)
            # Underfitting: test accuracy low
            elif current_test_acc < 0.8:
                model = under_or_overfitting(model, "underfitting", model_name)
            else:
                break
            
            model.fit(X_train_scaled, y_train)
            y_train_pred = model.predict(X_train_scaled)
            y_test_pred = model.predict(X_test_scaled)
            current_train_acc = accuracy_score(y_train, y_train_pred)
            current_test_acc = accuracy_score(y_test, y_test_pred)
            print(f"Iteration {iteration+1} for {model_name}: Train Accuracy={current_train_acc:.4f}, Test Accuracy={current_test_acc:.4f}")
            
            if current_train_acc <= old_train_acc and current_test_acc <= old_test_acc:
                improved = False
            iteration += 1
        
        # Save updated model and re-evaluate
        evaluate_model(y_train, y_train_pred, y_test, y_test_pred, model_name, model)

# Run hyperparameter tuning for all models (except the NN which uses a separate process)
hyperparameter_tuning(metrics_train, metrics_test, models, models_name)

## Graphing the Metrics

In [ ]:
def graph_models(metrics_train, metrics_test, models_name):
    models_to_graph = models_name.copy()
    metrics_list = ["accuracy", "precision", "recall", "f1_score"]
    colors = ["blue", "green", "orange", "red", "purple", "brown", "pink", "gray", "cyan", "magenta", "yellow"]
    
    num_metrics = len(metrics_list)
    num_models = len(models_to_graph)
    width = 0.15  
    group_spacing = 2.9
    x = np.arange(num_metrics) * group_spacing

    # Training Metrics
    print("Training Metrics:", metrics_train)
    plt.figure(figsize=(12, 6))
    for i, model in enumerate(models_to_graph):
        values = [metrics_train[f"{metric}_{model}"] for metric in metrics_list]
        bar_positions = x + i * width
        plt.bar(bar_positions, values, width, label=model.replace("_", " ").title(), color=colors[i % len(colors)])
    plt.xlabel('Metrics')
    plt.ylabel('Scores')
    plt.title('Training Metrics Comparison')
    plt.xticks(x + (width * (num_models - 1)) / 2, [m.title() for m in metrics_list])
    plt.ylim(0, 1)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Testing Metrics
    print("Testing Metrics:", metrics_test)
    plt.figure(figsize=(12, 6))
    for i, model in enumerate(models_to_graph):
        values = [metrics_test[f"{metric}_{model}"] for metric in metrics_list]
        bar_positions = x + i * width
        plt.bar(bar_positions, values, width, label=model.replace("_", " ").title(), color=colors[i % len(colors)])
    plt.xlabel('Metrics')
    plt.ylabel('Scores')
    plt.title('Testing Metrics Comparison')
    plt.xticks(x + (width * (num_models - 1)) / 2, [m.title() for m in metrics_list])
    plt.ylim(0, 1)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # KFold Metrics
    plt.figure(figsize=(12, 6))
    for i, model in enumerate(models_to_graph):
        values = [metrics_test[f"{metric}_kfold_{model}"] for metric in metrics_list]
        bar_positions = x + i * width
        plt.bar(bar_positions, values, width, label=model.replace("_", " ").title(), color=colors[i % len(colors)])
    plt.xlabel('Metrics')
    plt.ylabel('Scores')
    plt.title('KFold Metrics Comparison')
    plt.xticks(x + (width * (num_models - 1)) / 2, [m.title() for m in metrics_list])
    plt.ylim(0, 1)
    plt.legend()
    plt.tight_layout()
    plt.show()

graph_models(metrics_train, metrics_test, models_name)